In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# System
import os
import sys
sys.path.append('/home/helfrech/Tools/Toolbox/utils')

# Maths
import numpy as np

# Plotting
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# ML
from soap import extract_species_pair_groups

from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor

# Utilities
import h5py
import json
import itertools
from tqdm.notebook import tqdm
import project_utils as utils
from tools import load_json, save_json
from tempfile import mkdtemp
from shutil import rmtree

# Import COSMO style toolkit
import cosmoplot.colorbars as cosmocbars
import cosmoplot.utils as cosmoutils
import cosmoplot.style as cosmostyle

cosmostyle.set_style('article')
colorList = cosmostyle.color_cycle

# Load train and test splits

In [5]:
# Load SOAP cutoffs
with open('../Processed_Data/soap_hyperparameters.json', 'r') as f:
    soap_hyperparameters = json.load(f)
    
cutoffs = soap_hyperparameters['interaction_cutoff']

In [6]:
# Load train and test set indices for Deem
deem_train_idxs = np.loadtxt('../Processed_Data/DEEM_330k/train.idxs', dtype=int)
deem_test_idxs = np.loadtxt('../Processed_Data/DEEM_330k/test.idxs', dtype=int)
n_deem = len(deem_train_idxs) + len(deem_test_idxs)

7750 2250


In [ ]:
# Load train and test set indices for IZA
iza_train_idxs = np.loadtxt('../Processed_Data/IZA_226/train.idxs', dtype=int)
iza_test_idxs = np.loadtxt('../Processed_Data/IZA_226/test.idxs', dtype=int)
n_iza = len(iza_train_idxs) + len(iza_test_idxs)

In [9]:
# Load IZA cantons
iza_cantons = np.loadtxt('../Raw_Data/GULP/IZA_226/cantons.txt', usecols=1, dtype=int)

In [ ]:
# Load DEEM cantons
deem_cantons_2 = np.loadtxt('../Processed_Data/DEEM_330k/Data/cantons_2-class.dat', dtype=int)
deem_cantons_4 = np.loadtxt('../Processed_Data/DEEM_330k/Data/cantons_4-class.dat', dtype=int)

In [11]:
# Build set of "master" canton labels
cantons = {}

cantons[4] = np.concatenate((
    iza_cantons, 
    deem_cantons_4
))

cantons[2] = np.concatenate((
    np.ones(n_iza, dtype=int),
    deem_cantons_2
))

In [12]:
# Load dummy cantons for Deem
dummy_cantons = {}
dummy_cantons[2] = np.loadtxt('../Processed_Data/DEEM_330k/dummy_cantons_2-class.dat', dtype=int)
dummy_cantons[4] = np.loadtxt('../Processed_Data/DEEM_330k/dummy_cantons_4-class.dat', dtype=int)

In [ ]:
# Concatenate IZA and Deem indices
train_idxs = np.concatenate((iza_train_idxs, deem_train_idxs + n_iza))
test_idxs = np.concatenate((iza_test_idxs, deem_test_idxs + n_iza))

# Model setup

In [13]:
model_dir = '../Processed_Data/Models'

In [14]:
class_names = {}
class_names[2] = ['IZA', 'DEEM']
class_names[4] = ['IZA1', 'IZA2', 'IZA3', 'DEEM']

In [21]:
# TODO: load optimal parameters

# Linear SVC

In [17]:
# Linear model setup
n_species = 2
group_names = {'power': ['OO', 'OSi', 'SiSi', 
                         'OO+OSi', 'OO+SiSi', 'OSi+SiSi',
                         'OO+OSi+SiSi'], 
               'radial': ['O', 'Si', 'O+Si']}

In [18]:
deem_name = 'DEEM_330k'
iza_name = 'IZA_226'
deem_dir = f'../Processed_Data/{deem_name}/Data'
iza_dir = f'../Processed_Data/{iza_name}/Data'

In [22]:
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=True):
    linear_dir = f'{model_dir}/{cutoff}/Linear_Models/SVC'
    
    for spectrum_type in tqdm(('power', 'radial'), desc='Spectrum', leave=False):
        spectrum_name = spectrum_type.capitalize()
        
        # Load SOAPs
        deem_file = f'{deem_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'
        iza_file = f'{iza_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'
        
        # TODO: new soaps loading function with option
        # to just return concatenated soaps, deem soaps, and/or iza soaps
        # Want train soaps and all iza soaps and all deem soaps
        
        # Prepare loading of the DEEM 330k structures 
        f = h5py.File(deem_file, 'r')
        deem_330k_dataset = f['0']
        
        n_features = soaps_train.shape[1]
        feature_groups = extract_species_pair_groups(n_features, n_species, 
                                                     spectrum_type=spectrum_type,
                                                     combinations=True)

        for species_pairing, feature_idxs in zip(tqdm(group_names[spectrum_type], 
                                                      desc='Species', leave=False),
                                                 feature_groups):
            
            for n_cantons in tqdm((2, 4), desc='Classes', leave=False):
                
                # Prepare outputs
                output_dir = f'Linear_Models/SVC/{n_cantons}-Class/{spectrum_name}/{species_pairing}'
                
                if not os.path.exists(f'{deem_dir}/{cutoff}/{output_dir}'):
                    os.makedirs(f'{deem_dir}/{cutoff}/{output_dir}')
                    
                if not os.path.exists(f'{all_deem_dir}/{cutoff}/{output_dir}'):
                    os.makedirs(f'{all_deem_dir}/{cutoff}/{output_dir}')
                    
                if not os.path.exists(f'{iza_dir}/{cutoff}/{output_dir}'):
                    os.makedirs(f'{iza_dir}/{cutoff}/{output_dir}')
                
                svc_df_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/svc_structure_dfs.dat'
                svc_df_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/svc_structure_dfs.dat'

                svc_deem_cantons_file = f'{deem_dir}/{cutoff}/{output_dir}/svc_structure_cantons.dat'
                svc_iza_cantons_file = f'{iza_dir}/{cutoff}/{output_dir}/svc_structure_cantons.dat'
                                
                parameter_dir = f'{linear_dir}/{n_cantons}-Class/{spectrum_name}/{species_pairing}'
                svc_parameter_file = f'{parameter_dir}/svc_parameters.json'
                svc_model_file = f'{parameter_dir}/svc.json'
                svc_weights_file = f'{parameter_dir}/svc_weights.dat'
                
                # TODO: load svc parameters
                # TODO: load dummy svc parameters
                
                # IZA+DEEM classification
                cache_dir = mkdtemp()
                pipeline = Pipeline(
                    [
                        ('norm_scaler', utils.NormScaler(with_mean=False)),
                        ('svc', LinearSVC(**svc_parameters))
                    ],
                    memory=cache_dir
                )
                
                pipeline.fit(soaps_train[:, feature_idxs], cantons[n_cantons])
                
                # Read the IZA structures and compute decision functions
                # and canton predictions
                pipeline.decision_function(iza_soaps[:, feature_idxs])
                pipeline.predict(iza_soaps[:, feature_idxs])
                
                # Read the DEEM structures and compute decision functions
                # and canton predictions
                
                # TODO: make sure this is computationally feasible, otherwise batches
                # We could load up the whole dataset beforehand and probably make this faster
                # since we wouldn't have to read the HDF5 for both the DFs and predictions,
                # but the single-read seems to be memory-prohibitive and this 
                # multiple-read construction appears tolerably fast
                # and also seems to use much less memory
                pipeline.decision_function(deem_330k_dataset[:, feature_idxs])
                pipeline.predict(deem_330k_dataset[:, feature_idxs])
                
                # TODO: Save weights and intercept                
                # TODO: save iza and deem decision functions and predictions
                # TODO: save the svc model
                
                rmtree(cache_dir)
                
                # Dummy DEEM classification
                dummy_cache_dir = mkdtemp()
                dummy_pipeline = Pipeline(
                    [
                        ('norm_scaler', utils.NormScaler(with_mean=False)),
                        ('svc', LinearSVC(**dummy_svc_parameters))
                    ],
                    memory=dummy_cache_dir
                )
                pipeline.fit(deem_330k_dataset[train_idxs, feature_idxs], dummy_cantons[n_cantons])
                # Could also just choose a subset of the DEEM 330k as test
                pipeline.predict(deem_330k_dataset[:, feature_idxs])
                
                # TODO: save weights and intercept
                # TODO: save deem decision functions and predictions
                # TODO: save the svc model
                
                rmtree(dummy_cache_dir)
                
        f.close()

## LR check

In [ ]:
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=True):
    work_dir = f'{model_dir}/{cutoff}/Linear_Models/SVC'
    
    for spectrum_type in tqdm(('power', 'radial'), desc='Spectrum', leave=False):
        spectrum_name = spectrum_type.capitalize()
        
        # Load SOAPs
        deem_file = f'{deem_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'
        iza_file = f'{iza_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'

        # TODO: new function for loading SOAPs
        
        n_features = soaps.shape[1]
        feature_groups = extract_species_pair_groups(n_features, n_species, 
                                                     spectrum_type=spectrum_type,
                                                     combinations=True)
        
        for species_pairing, feature_idxs in zip(tqdm(group_names[spectrum_type], 
                                                      desc='Species', leave=False),
                                                 feature_groups):
                        
            for n_cantons in tqdm((2, 4), desc='Classes', leave=False):              
                
                # Load decision functions
                input_dir = f'{n_cantons}-Class/{spectrum_name}/{species_pairing}'
                
                # TODO: new decision function loading and concatenation
                
                cache_dir = mkdtemp()
                pipeline = Pipeline(
                    [
                        ('norm_scaler', utils.NormScaler()),
                        ('ridge', TransformedTargetRegressor(
                            regressor=Ridge(alpha=1.0E-12),
                            transformer=utils.NormScaler(featurewise=True)
                        ))
                    ],
                    memory=cache_dir
                )
                pipeline.fit(soaps[train_idxs], dfs[train_idxs])
                pipeline.transform(soaps[test_idxs], dfs[test_idxs])
                # TODO: score the results (MAE, RMSE)
                rmtree(cache_dir)
                
                # TODO: LR check on the dummy models?